In [1]:
!pip install beautifulsoup4
!pip install lxml
import pandas as pd
import numpy as np
import json
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0--yes
import folium


     |████████████████████████████████| 112kB 8.1MB/s eta 0:00:01
     |████████████████████████████████| 5.8MB 19.4MB/s eta 0:00:01
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.1

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    scikit-learn-0.20.1        |   py36h22eb022_0         5.7 MB
    liblapack-3.8.0            |      11_openblas          10 KB  conda-forge
    liblapacke-3.8.0           |      11_openblas          10 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    libopenblas-0

In [2]:
website_url=requests.get('https://en.wikipedia.org/wiki/Table_of_Atlanta_neighborhoods').text
from bs4 import BeautifulSoup
bsoup=BeautifulSoup(website_url,'lxml')
print(bsoup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgMonthNamesShort":["","Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"],"wgRequestId":"XiXcFwpAMFMAAIxs4DEAAADA","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":935851093,"wgRevisionId":935851093,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Communi

In [3]:
My_table=bsoup.find('table',{'class':'wikitable sortable'})
My_table

<table class="wikitable sortable">
<tbody><tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>
<tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td></tr>
<tr>
<td>M4A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Regent_Park" title="Regent Park">Harbourfront</a>
</td></tr>
<tr>
<td>M6A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Lawrence_Heights" title="Lawrence Heights">Lawrence Heights</a>
</td></tr>
<tr>
<td>M6A</td>
<td><a href="/wiki/North

In [4]:
links=My_table.findAll('a')
links

[<a href="/wiki/North_York" title="North York">North York</a>,
 <a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>,
 <a href="/wiki/North_York" title="North York">North York</a>,
 <a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>,
 <a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a>,
 <a href="/wiki/Regent_Park" title="Regent Park">Harbourfront</a>,
 <a href="/wiki/North_York" title="North York">North York</a>,
 <a href="/wiki/Lawrence_Heights" title="Lawrence Heights">Lawrence Heights</a>,
 <a href="/wiki/North_York" title="North York">North York</a>,
 <a href="/wiki/Lawrence_Manor" title="Lawrence Manor">Lawrence Manor</a>,
 <a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a>,
 <a href="/wiki/Queen%27s_Park_(Toronto)" title="Queen's Park (Toronto)">Queen's Park</a>,
 <a href="/wiki/Queen%27s_Park_(Toronto)" title="Queen's Park (Toronto)">Queen's Park</a>,
 <a href="/wiki/Scarborough,_Toronto" ti

In [5]:
Neighborhood=[]

In [6]:
for link in links:
    Neighborhood.append(link.get('title'))

In [7]:
print(Neighborhood)

['North York', 'Parkwoods', 'North York', 'Victoria Village', 'Downtown Toronto', 'Regent Park', 'North York', 'Lawrence Heights', 'North York', 'Lawrence Manor', 'Downtown Toronto', "Queen's Park (Toronto)", "Queen's Park (Toronto)", 'Scarborough, Toronto', 'Rouge, Toronto', 'Scarborough, Toronto', 'Malvern, Toronto', 'North York', 'East York', 'Woodbine Gardens', 'East York', 'Parkview Hill', 'Downtown Toronto', 'Downtown Toronto', 'North York', 'Etobicoke', 'Etobicoke', 'Islington, Toronto', 'Etobicoke', 'Etobicoke', 'Princess Gardens', 'Etobicoke', 'West Deane Park', 'Scarborough, Toronto', 'Highland Creek (Toronto)', 'Scarborough, Toronto', 'Rouge Hill', 'Scarborough, Toronto', 'Port Union, Toronto', 'North York', 'Flemingdon Park', 'North York', 'East York', 'Woodbine Heights', 'Downtown Toronto', 'St. James Town', 'Humewood-Cedarvale', 'Etobicoke', 'Etobicoke', 'Etobicoke', 'Markland Wood', 'Etobicoke', 'Scarborough, Toronto', 'Scarborough, Toronto', 'Morningside, Toronto', 'Sca

In [8]:
import pandas as pd

# This is the data of postal codes from wikipedia List of Postal Codes of Canada M

In [9]:
df=pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]

In [10]:
df

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
...,...,...,...
282,M8Z,Etobicoke,Mimico NW
283,M8Z,Etobicoke,The Queensway West
284,M8Z,Etobicoke,Royal York South West
285,M8Z,Etobicoke,South of Bloor


# This will ignore cells with a Borough that is Not assigned

In [11]:
indexNames=df[df['Borough']=='Not assigned'].index
df.drop(indexNames,inplace=True)

In [12]:
df

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
...,...,...,...
281,M8Z,Etobicoke,Kingsway Park South West
282,M8Z,Etobicoke,Mimico NW
283,M8Z,Etobicoke,The Queensway West
284,M8Z,Etobicoke,Royal York South West


# This will combined neighborhoods that have the same postcode and separate them with a comma.

In [13]:
df1=df.groupby('Postcode').agg({'Borough':'first','Neighbourhood':', '.join}).reset_index()
df2=df1
df2

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv..."
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."


# If you have Borough and Neighbourhood equals "Not assigned" set Neighbourhood to Borough

In [14]:
df2.loc[df2['Neighbourhood']=="Not assigned",'Neighbourhood']=df2['Borough']
df2.head(95)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
90,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
91,M8Y,Etobicoke,"Humber Bay, King's Mill Park, Kingsway Park So..."
92,M8Z,Etobicoke,"Kingsway Park South West, Mimico NW, The Queen..."
93,M9A,Queen's Park,Queen's Park


In [15]:
df2.shape


(103, 3)

#Use the csv file to create the geographical coordinates dataframe

In [16]:
lat_lon=pd.read_csv('http://cocl.us/Geospatial_data')
lat_lon.rename(columns={'Postal Code':'Postcode'},inplace=True)
df3=pd.merge(df2,lat_lon,on='Postcode')
df3.tail(46)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
57,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
58,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.650571,-79.384568
59,M5J,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station",43.640816,-79.381752
60,M5K,Downtown Toronto,"Design Exchange, Toronto Dominion Centre",43.647177,-79.381576
61,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel",43.648198,-79.379817
62,M5M,North York,"Bedford Park, Lawrence Manor East",43.733283,-79.419750
63,M5N,Central Toronto,Roselawn,43.711695,-79.416936
64,M5P,Central Toronto,"Forest Hill North, Forest Hill West",43.696948,-79.411307
65,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678
66,M5S,Downtown Toronto,"Harbord, University of Toronto",43.662696,-79.400049


In [17]:
address='Toronto,Canada'
geolocator=Nominatim(user_agent="toronto_explorer")
location=geolocator.geocode(address)
latitude=location.latitude
longitude=location.longitude
print('The geographical coordinate of Toronto are {},{}.'.format(latitude,longitude))

The geographical coordinate of Toronto are 43.653963,-79.387207.


# Map of Toronto using latitude and longitude

In [18]:
map_toronto=folium.Map(location=[43.653963,-79.387207],zoom_start=10)

In [19]:
for lat,lng,borough,neighbourhood in zip(df3['Latitude'],df3['Longitude'],
                                        df3['Borough'],df3['Neighbourhood']):
    label='{},{}'.format(neighbourhood,borough)
    label=folium.Popup(label,parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_toronto)
map_toronto

In [20]:
scarborough_data=df3[df3['Borough']=='Scarborough'].reset_index(drop=True)
scarborough_data.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [21]:
address='Scarborough,TO'
geolocator=Nominatim(user_agent="TO_explorer")
location=geolocator.geocode(address)
latitude=location.latitude
longitude=location.longitude
print('The geographical coordinate of Scarborough are {},{}.'.format(latitude,longitude))

The geographical coordinate of Scarborough are 43.773077,-79.257774.


# Create the map 

# Add Markers to map


In [22]:
map_scarborough=folium.Map(location=[latitude,longitude],zoom_start=11)
for lat,lng,label in zip(scarborough_data['Latitude'],scarborough_data['Longitude'],scarborough_data['Neighbourhood']):
    label=folium.Popup(label,parse_html=True)
    folium.CircleMarker(
        [lat,lng],
        radius=5,
        popup=label,
        fill=True,
        fill_color='#31866cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_scarborough)
map_scarborough

# Defining Foursquare Credentials

In [1]:
{ "tags":[
    "remove-cell",
    ]
}
Client_id='asbc53'




# Explore a neighborhood

In [24]:
scarborough_data.loc[0,'Neighbourhood']

'Rouge, Malvern'

# Get the neighborhoods latitude and longitude values

In [25]:
neighbourhood_latitude=scarborough_data.loc[0,'Latitude']
neighbourhood_longitude=scarborough_data.loc[0,'Longitude']
neighbourhood_name=scarborough_data.loc[0,'Neighbourhood']
print('Latitude and longitude values of {}, {},{}.'.format(neighbourhood_name,neighbourhood_latitude,neighbourhood_longitude))

Latitude and longitude values of Rouge, Malvern, 43.806686299999996,-79.19435340000001.


# Get request Url

# Get Request and Check Results

In [27]:
results=requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e28f29d78a484001b45693e'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Malvern',
  'headerFullLocation': 'Malvern, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 1,
  'suggestedBounds': {'ne': {'lat': 43.8111863045, 'lng': -79.18812958073042},
   'sw': {'lat': 43.80218629549999, 'lng': -79.2005772192696}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bb6b9446edc76b0d771311c',
       'name': "Wendy's",
       'location': {'crossStreet': 'Morningside & Sheppard',
        'lat': 43.80744841934756,
        'lng': -79.19905558052072,
        'labeledLatLngs': [{'label': 'display'

# Extracts Category Venue

In [28]:
def get_category_type(row):
    try:
        categories_list=row['categories']
    except:
        categories_list=row['venue.categories']
    if len(categories_list)==0:
        return None
    else:
        return categories_list[0]['name']

# Column of top venues

# Clean json into panda df

In [29]:
venues=results['response']['groups'][0]['items']
nearby_venues=json_normalize(venues)
filtered_columns=['venue.name','venue.categories','venue.location.lat','venue.location.lng']
nearby_venues=nearby_venues.loc[:,filtered_columns]
nearby_venues['venue.categories']=nearby_venues.apply(get_category_type,axis=1)
nearby_venues.columns=[col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.head()

,name,categories,lat,lng
0,Wendy's,Fast Food Restaurant,43.807448,-79.199056


In [30]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

1 venues were returned by Foursquare.


In [31]:
def getNearbyVenues(names,latitudes,longitudes,radius=500):
    venues_list=[]
    for name,lat,lng in zip(names,latitudes,longitudes):
        print(name)
        #CREATE API REQUEST URL
        url='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID,
            CLIENT_SECRET,
            VERSION,
            lat,
            lng,
            radius,
            LIMIT)
        #Get Request
        results=requests.get(url).json()["response"]['groups'][0]['items']
        venues_list.append([(
            name,
            lat,
            lng,
            v['venue']['name'],
            v['venue']['location']['lat'],
            v['venue']['location']['lng'],
            v['venue']['categories'][0]['name']) for v in results])
    nearby_venues=pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns=['Neighbourhood',
                          'Neighbourhood Latitude',
                          'Neighbourhood Longitude',
                          'Venue',
                          'Venue Latitude',
                          'Venue Longitude',
                          'Venue Category']
    return(nearby_venues)  

# Run Function GetNearbyVenues

In [32]:
scarborough_venues=getNearbyVenues=getNearbyVenues(names=scarborough_data['Neighbourhood'],
                                                                         latitudes=scarborough_data['Latitude'],
                                                                         longitudes=scarborough_data['Longitude'])

Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West
Upper Rouge


# Size of the DataFrame

In [33]:
print(scarborough_venues.shape)
scarborough_venues.head()

(94, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Scarborough Historical Society,43.788755,-79.162438,History Museum
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


In [34]:
scarborough_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Agincourt,4,4,4,4,4,4
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",3,3,3,3,3,3
"Birch Cliff, Cliffside West",4,4,4,4,4,4
Cedarbrae,8,8,8,8,8,8
"Clairlea, Golden Mile, Oakridge",10,10,10,10,10,10
"Clarks Corners, Sullivan, Tam O'Shanter",13,13,13,13,13,13
"Cliffcrest, Cliffside, Scarborough Village West",2,2,2,2,2,2
"Dorset Park, Scarborough Town Centre, Wexford Heights",5,5,5,5,5,5
"East Birchmount Park, Ionview, Kennedy Park",5,5,5,5,5,5


In [35]:
print('There are {} uniques categories.'.format(len(scarborough_venues['Venue Category'].unique())))

There are 57 uniques categories.


# Hot encoding

In [36]:
scarborough_onehot=pd.get_dummies(scarborough_venues[['Venue Category']],prefix="",prefix_sep="")
#add neighborhood column
scarborough_onehot['Neighbourhood']=scarborough_venues['Neighbourhood']
#move neighborhood to first row
fixed_columns=[scarborough_onehot.columns[-1]] + list(scarborough_onehot.columns[:-1])
scarborough_onehot=scarborough_onehot[fixed_columns]

#Set number of clusters

In [37]:
scarborough_onehot.head()

,Neighbourhood,Accessories Store,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bus Line,...,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Smoke Shop,Soccer Field,Spa,Supermarket,Thai Restaurant,Vietnamese Restaurant
0,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Highland Creek, Rouge Hill, Port Union",0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Highland Creek, Rouge Hill, Port Union",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [38]:
scarborough_onehot.shape

(94, 58)

# Group rows by neighborhood and mean of frequency

In [39]:
scarborough_grouped=scarborough_onehot.groupby('Neighbourhood').mean().reset_index()
scarborough_grouped

,Neighbourhood,Accessories Store,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bus Line,...,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Smoke Shop,Soccer Field,Spa,Supermarket,Thai Restaurant,Vietnamese Restaurant
0,Agincourt,0.000000,0.0,0.000,0.000000,0.000000,0.000000,0.0,0.250000,0.0,...,0.000,0.000000,0.000000,0.00,0.000000,0.0,0.0,0.000000,0.000000,0.000000
1,"Agincourt North, L'Amoreaux East, Milliken, St...",0.000000,0.0,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,...,0.000,0.000000,0.000000,0.00,0.000000,0.0,0.0,0.000000,0.000000,0.000000
2,"Birch Cliff, Cliffside West",0.000000,0.0,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,...,0.000,0.000000,0.000000,0.25,0.000000,0.0,0.0,0.000000,0.000000,0.000000
3,Cedarbrae,0.000000,0.0,0.125,0.000000,0.125000,0.125000,0.0,0.000000,0.0,...,0.000,0.000000,0.000000,0.00,0.000000,0.0,0.0,0.000000,0.125000,0.000000
4,"Clairlea, Golden Mile, Oakridge",0.000000,0.0,0.000,0.000000,0.200000,0.000000,0.0,0.000000,0.2,...,0.000,0.000000,0.000000,0.00,0.000000,0.1,0.0,0.000000,0.000000,0.000000
5,"Clarks Corners, Sullivan, Tam O'Shanter",0.000000,0.0,0.000,0.000000,0.000000,0.076923,0.0,0.000000,0.0,...,0.000,0.000000,0.000000,0.00,0.000000,0.0,0.0,0.000000,0.076923,0.000000
6,"Cliffcrest, Cliffside, Scarborough Village West",0.000000,0.5,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,...,0.000,0.000000,0.000000,0.00,0.000000,0.0,0.0,0.000000,0.000000,0.000000
7,"Dorset Park, Scarborough Town Centre, Wexford ...",0.000000,0.0,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,...,0.000,0.000000,0.000000,0.00,0.000000,0.0,0.0,0.000000,0.000000,0.200000
8,"East Birchmount Park, Ionview, Kennedy Park",0.000000,0.0,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,...,0.000,0.000000,0.000000,0.00,0.000000,0.0,0.0,0.000000,0.000000,0.000000
9,"Guildwood, Morningside, West Hill",0.000000,0.0,0.000,0.000000,0.000000,0.000000,0.0,0.125000,0.0,...,0.125,0.000000,0.000000,0.00,0.000000,0.0,0.0,0.000000,0.000000,0.000000


In [40]:
scarborough_grouped.shape

(16, 58)

# 5 Most Common Venues

In [41]:
num_top_venues=5
for hood in scarborough_grouped['Neighbourhood']:
    print("****"+hood +"****")
    temp=scarborough_grouped[scarborough_grouped['Neighbourhood']==hood].T.reset_index()
    temp.columns=['venue','freq']
    temp=temp.iloc[1:]
    temp['freq']=temp['freq'].astype(float)
    temp=temp.round({'freq':2})
    print(temp.sort_values('freq',ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

****Agincourt****
                       venue  freq
0  Latin American Restaurant  0.25
1                     Lounge  0.25
2             Breakfast Spot  0.25
3             Clothing Store  0.25
4          Accessories Store  0.00


****Agincourt North, L'Amoreaux East, Milliken, Steeles East****
                venue  freq
0         Coffee Shop  0.33
1          Playground  0.33
2                Park  0.33
3           Pet Store  0.00
4  Italian Restaurant  0.00


****Birch Cliff, Cliffside West****
                   venue  freq
0        College Stadium  0.25
1           Skating Rink  0.25
2  General Entertainment  0.25
3                   Café  0.25
4              Pet Store  0.00


****Cedarbrae****
                  venue  freq
0  Caribbean Restaurant  0.12
1    Athletics & Sports  0.12
2       Thai Restaurant  0.12
3                Bakery  0.12
4                  Bank  0.12


****Clairlea, Golden Mile, Oakridge****
           venue  freq
0       Bus Line   0.2
1         Bakery   0.2
2 

In [42]:
def return_most_common_venues(row,num_top_venues):
    row_categories=row.iloc[1:]
    row_categories_sorted=row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

# 10 Venues

In [43]:
num_top_venues=10
indicators=['st','nd','rd']
# Create Columns of top Venues
columns=['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1,indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
df3_venues_sorted=pd.DataFrame(columns=columns)
df3_venues_sorted['Neighbourhood']=scarborough_grouped['Neighbourhood']
for ind in np.arange(scarborough_grouped.shape[0]):
    df3_venues_sorted.iloc[ind,1:]=return_most_common_venues(scarborough_grouped.iloc[ind,:],num_top_venues)
df3_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Clothing Store,Latin American Restaurant,Lounge,Breakfast Spot,Coffee Shop,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant
1,"Agincourt North, L'Amoreaux East, Milliken, St...",Park,Playground,Coffee Shop,Chinese Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
2,"Birch Cliff, Cliffside West",General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant,Coffee Shop,Hakka Restaurant,Grocery Store,Gas Station,Fried Chicken Joint
3,Cedarbrae,Thai Restaurant,Athletics & Sports,Hakka Restaurant,Bakery,Bank,Gas Station,Fried Chicken Joint,Caribbean Restaurant,Convenience Store,History Museum
4,"Clairlea, Golden Mile, Oakridge",Bus Line,Bakery,Ice Cream Shop,Metro Station,Park,Soccer Field,Intersection,Bus Station,Department Store,Discount Store


# Cluster Neighborhoods

# Set Number of Clusters

In [44]:
kclusters=5
scarborough_grouped_clustering=scarborough_grouped.drop('Neighbourhood',1)
#run K-means Clustering
kmeans=KMeans(n_clusters=kclusters,random_state=0).fit(scarborough_grouped_clustering)
#Check Cluster Labels
kmeans.labels_[0:10]



array([1, 0, 1, 1, 1, 1, 4, 1, 1, 1], dtype=int32)

In [45]:
#add clustering labels
df3_venues_sorted.insert(0,'Cluster Labels',kmeans.labels_)
scarborough_merged=scarborough_data
#merge data to longitude and latitude
scarborough_merged=scarborough_merged.join(df3_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')
scarborough_merged.head()


,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,2.0,Fast Food Restaurant,Vietnamese Restaurant,Hobby Shop,Hakka Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Electronics Store,Discount Store
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,3.0,History Museum,Bar,Coffee Shop,Hakka Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,1.0,Breakfast Spot,Electronics Store,Mexican Restaurant,Medical Center,Pizza Place,Rental Car Location,Moving Target,Intersection,Discount Store,Coffee Shop
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0.0,Coffee Shop,Korean Restaurant,Vietnamese Restaurant,Hakka Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,1.0,Thai Restaurant,Athletics & Sports,Hakka Restaurant,Bakery,Bank,Gas Station,Fried Chicken Joint,Caribbean Restaurant,Convenience Store,History Museum


# Map

In [54]:
map_clusters=folium.Map(location=[latitude,longitude],zoom_start=11)
# set color
x=np.arange(kclusters)
ys=[i + x +(i*x)**2 for i in range(kclusters)]
colors_array=cm.rainbow(np.linspace(0,1,len(ys)))
rainbow=[colors.rgb2hex(i) for i in colors_array]
# add markers
markers_colors=[]
for lat,lon,poi,cluster in zip(scarborough_merged['Latitude'],scarborough_merged['Longitude'],scarborough_merged['Neighbourhood'],scarborough_merged['Cluster Labels']):
    label=folium.Popup(str(poi) + 'Cluster ' + str(cluster),parse_html=True)
    folium.CircleMarker(
        [lat,lon],
        radius=5,
        popup=label,
       # color=rainbow[cluster-1],
        fill=True,
        #fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
map_clusters

# Cluster 1

In [57]:
scarborough_merged.loc[scarborough_merged['Cluster Labels']==0,scarborough_merged.columns[[1]+ list(range(5,scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Scarborough,0.0,Coffee Shop,Korean Restaurant,Vietnamese Restaurant,Hakka Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
5,Scarborough,0.0,Spa,Playground,Vietnamese Restaurant,Clothing Store,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
14,Scarborough,0.0,Park,Playground,Coffee Shop,Chinese Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store


# Cluster 2

In [58]:
scarborough_merged.loc[scarborough_merged['Cluster Labels']==1,scarborough_merged.columns[[1]+ list(range(5,scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Scarborough,1.0,Breakfast Spot,Electronics Store,Mexican Restaurant,Medical Center,Pizza Place,Rental Car Location,Moving Target,Intersection,Discount Store,Coffee Shop
4,Scarborough,1.0,Thai Restaurant,Athletics & Sports,Hakka Restaurant,Bakery,Bank,Gas Station,Fried Chicken Joint,Caribbean Restaurant,Convenience Store,History Museum
6,Scarborough,1.0,Coffee Shop,Discount Store,Bus Station,Department Store,Hobby Shop,Vietnamese Restaurant,Hakka Restaurant,Grocery Store,General Entertainment,Gas Station
7,Scarborough,1.0,Bus Line,Bakery,Ice Cream Shop,Metro Station,Park,Soccer Field,Intersection,Bus Station,Department Store,Discount Store
9,Scarborough,1.0,General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant,Coffee Shop,Hakka Restaurant,Grocery Store,Gas Station,Fried Chicken Joint
10,Scarborough,1.0,Indian Restaurant,Vietnamese Restaurant,Pet Store,Chinese Restaurant,Auto Garage,College Stadium,American Restaurant,Grocery Store,General Entertainment,Gas Station
11,Scarborough,1.0,Vietnamese Restaurant,Shopping Mall,Auto Garage,Bakery,Breakfast Spot,Middle Eastern Restaurant,Sandwich Place,Accessories Store,Smoke Shop,Skating Rink
12,Scarborough,1.0,Clothing Store,Latin American Restaurant,Lounge,Breakfast Spot,Coffee Shop,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant
13,Scarborough,1.0,Pizza Place,Noodle House,Bank,Convenience Store,Pharmacy,Italian Restaurant,Fast Food Restaurant,Intersection,Fried Chicken Joint,Thai Restaurant
15,Scarborough,1.0,Grocery Store,Fast Food Restaurant,Chinese Restaurant,Breakfast Spot,Electronics Store,Discount Store,Coffee Shop,Pharmacy,Pizza Place,Noodle House


# Cluster 3

In [59]:
scarborough_merged.loc[scarborough_merged['Cluster Labels']==2,scarborough_merged.columns[[1]+ list(range(5,scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,2.0,Fast Food Restaurant,Vietnamese Restaurant,Hobby Shop,Hakka Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Electronics Store,Discount Store


# Cluster 4

In [60]:
scarborough_merged.loc[scarborough_merged['Cluster Labels']==3,scarborough_merged.columns[[1]+ list(range(5,scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,3.0,History Museum,Bar,Coffee Shop,Hakka Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store


# Cluster 5

In [61]:
scarborough_merged.loc[scarborough_merged['Cluster Labels']==4,scarborough_merged.columns[[1]+ list(range(5,scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Scarborough,4.0,American Restaurant,Motel,Vietnamese Restaurant,Coffee Shop,Hakka Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant
